In [ ]:
import seaborn as sns
sns.set()

# Optimization in Python

Imagine that we have a quantity we want to maximize (such as revenue) or minimize (some cost, an error rate, etc) while we're subject to some form of constraint, such as a limit on how many of something we can produce or a minimum payment etc. Such a problem can be solved using optimization. 

There are a variety of techniques for solving these problems and a variety of Python tools that implement them. We will focus on using [SciPy](https://docs.scipy.org/doc/scipy/tutorial/general.html) and we'll look at the following three types of problems:

- Simple minimization
- Linear programming
- Nonlinear constrained minimization


## Simple minimization

Let's start with an example where our optimization problem is not subject to any constraints. This means we have an equation that we just need to find the minimum (or maximum) of and in this case there are many tools available.  We'll look at one in SciPy here.

As an extremely simple example, we may be in a situation where we gain a profit from doing something, but the more we do it, the more the associated cost grows.  This might represent someone's output as they work more hours, while the increasing cost is the increasing chance they'll make a costly mistake as they get tired.

We'll model this very simply as

$$p = x - \frac{1}{15}x^2$$

Here we are creating a **decision variable** and an **objective**.  The objective here is $p$, our profit - this is the thing we're seeking to maximize or minimize.  The thing we are able to change is $x$, the number of hours worked. This is our decision variable - we are deciding how many hours to work.  For all of our problems, we'll need to identify these to be able to make a model.

In [ ]:
def profit(x):
    return x - x**2 / 15

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0, 10, 100)
plt.plot(x, profit(x))
plt.xlabel('x (the number of hours worked)')
plt.ylabel('p (profit)');

By eye, it looks like we reach the maximum somewhere around 7, but we can be much more precise.  

This example equation we want to minimize is so simple that we can easily compute the root of the derivative analytically.

When that is not the case, we can solve this optimization problem numerically by using the `minimize` routine in SciPy. We need only provide it the function and a rough guess as to where the function is minimal. Note that this is the `minimize` routine, so we'll actually need to provide it with the negative of our function given that we are looking for the maximum.

In [ ]:
from scipy.optimize import minimize

minimize(lambda x: -profit(x), x0=7)

So we see we get a maximum at 7.5 (hours, in our example), producing 3.75 units of profit.

## Linear programming

Not all problems are quite so simple.  Often, we find that we have some constraints - variables must be positive, or combinations of variables must sum to particular values or be less than certain values. If the equations that we make out of our decision variables are linear, then we can use linear programming.

Here we mean linear in the mathematical sense

$$1.5 x + 2.3 y + 4.5 z = 22$$

where all our variables enter with constant coefficients. On the ther hand

$$x y = 45$$

would be nonlinear, and we would handle that with a different technique.

Let's set up a fairly simple situation to try this out.


### Factory output example


We have a factory that produces two kinds of products, call them $P_1$ and $P_2$.  We make a profit of 5 when we sell one pound of $P_1$, and a profit of 8 when we sell a pound of $P_2$.  Making a pound of $P_1$ uses two units of a raw resource, and a pound of $P_2$ uses 3.  We're limited to making 40 pounds in total, and can only get 100 units of resources a day.  How many pounds of each product should we make?

We need to figure out three things:
1. What are our decision variables?
1. What are we seeking to maximize?
1. What are our constraints?


For our decision variables, there are only two: how many pounds of $P_1$'s and how many of $P_2$'s.  We'll just call those $P_1$ and $P_2$.

We're seeking to maximize profit, so we write that as:

$$p = 5 P_1 + 8 P_2$$

Finally, we are constrained both by total output and resources. That gives us two equations:

$$P_1 + P_2 \leq 40$$
$$2 P_1 + 3 P_2 \leq 100$$

We also can't produce a _negative_ number of pounds, so we also have

$$P_1 \geq 0$$
$$P_2 \geq 0$$

Now we're ready.  We will use [`linprog` from SciPy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html) to solve this problem.

In [ ]:
from scipy.optimize import linprog

c = [-5, -8]   # negative signs because linprog minimizes our objective
A = [[1, 1], [2, 3]]
b = [40, 100]
p1_bounds = (0, np.inf)
p2_bounds = (0, np.inf)
res = linprog(c, A_ub=A, b_ub=b, bounds=[p1_bounds, p2_bounds])

In [ ]:
res.message

In [ ]:
print(f'A={round(res.x[0], 3)}, B={round(res.x[1], 3)}')

Unsurprisingly, we should just make as many of the higher profit margin ones as we can.

This is a fairly common outcome in simple linear systems - we will always wind up on one of the constraints.

We can visualize this - we'll color in the region that violates the inequality $P_1 + P_2 \leq 40$ in red, and the region that violates $2P_1 + 3P_2 \leq 100$ in blue.  We are then only allowed to find values in the white region left, and the algorithm will find the one with the highest value of our objective.  We'll mark that with a dot.

In [ ]:
x = np.linspace(0,40,100)
plt.fill_between(x,40 - x, 40, color='red', alpha=0.2, label="$P_1 + P_2 > 40$")
plt.fill_between(x,(100 - 2*x)/3, 40, alpha=0.2, label="$2P_1 + 3P_2 > 100$")
plt.plot([res.x[0]], [res.x[1]],'o', label='solution')
plt.legend(loc='lower left')
plt.xlabel("$P_1$")
plt.ylabel("$P_2$")
plt.title("Factory output constraints");

## Nonlinear optimization

Unfortunately, our models aren't always nicely described by linear models. In those situations, where either our objective function or our constraints are not linear, we need to use nonlinear methods. There are a huge variety of them, but most of them are actually available as options in the `minimize` function in SciPy that we saw earlier.

Let's demonstrate this with a simple textbook problem.

To keep it simple, let's minimize the cost of building a fence around a rectangular field, where the "vertical" and "horizontal" sides cost different amounts (say 4/unit and 7/unit), but we want to enclose at least 100 units of area.

Our objective is fairly simple, we want to minimize:

$$c = 4\cdot 2 \cdot y + 7\cdot 2 \cdot x$$

Our constraint is:

$$ x\:y \geq 100$$

To transform this into code, we need to first choose a solver.  There are several for nonlinear problems with constraints, described [here](https://docs.scipy.org/doc/scipy/tutorial/optimize.html). We'll choose `SLSQP` (this is "sequential least squares programming") for convenience.  

We have an inequality constraint, which the system requires to be of the form:

$$f(x,y) \geq 0$$

Rewriting ours in that form gives us:

$$x\:y - 100 \geq 0$$

We also have to compute the Jacobian (the derivative) for our constraint.  We'll do this by hand and get

$$\begin{bmatrix}y & x\end{bmatrix}$$

Now we package that together into a dictionary and we are ready. Note that we set `type` to `ineq` to indicate an inequality constraint.  If we had an equality constraint, say $x\:y = 100$ instead, we'd set it to `eq`.  If we have both, we make a dictionary for each and pass it to `minimize` as `constraints=[eq_cons, ineq_cons]`.

We also want to bound our values, since we can only have positive lengths of fence.  We don't have a constraint in the other direction (we presumably can make the area as long and skinny as we like), so we set the upper bound to `np.inf` for infinity.

Finally, we have to have an initial guess. This is actually fairly important, since the method can find a local minimum rather than the global minimum.  If we are concerned that is happening, we can either try starting at a variety of points, or we can switch to a different minimization package.

In [ ]:
def cost(xy):
    return 7*2*xy[0] + 4*2*xy[1]

def area(xy):
    return xy[0]*xy[1]

ineq_cons = {'type': 'ineq', 
             'fun': lambda xy: [area(xy) - 100],
             'jac': lambda xy: [[xy[1], xy[0]]]} # we provide the derivates of constraints

results = minimize(cost, [1, 1], 
                   method='SLSQP', 
                   constraints=ineq_cons, 
                   bounds=[(0,np.inf),(0,np.inf)])

print('x = {:.3f}, y = {:.3f}, giving cost = {:.3f} and area = {:.3f}'
      .format(results.x[0], results.x[1], cost(results.x), area(results.x)))

Again, we find that we are right at the minimum allowable area.  This doesn't always happen with nonlinear systems the way it does with linear ones, but is relatively common.

### Portfolio example

Let's try a slightly more realistic situation. We want to create an investment portfolio, maximizing returns and minimizing risk, say among three stocks. Risk is traditionally modeled with the variance, which takes into account both individual volatility and correlations between stocks.

We won't worry about the correlations for the moment, instead we'll look at a simplified model where we have each stock's individual variance, and a term for the overall market.

$$c = \sum_i r_i s_i - \gamma \left(\sum_i v_i s_i^2 + v_{total}(s_1 + s_2 + s_3)^2\right)$$

Where $s_i$ is how much we're investing in stock $i$, $r_i$ is the expected return for stock $i$, $v_i$ is the stock's variance, and $v_{total}$ is our overall market variance.

We presumably want an upper limit on our investment, giving us a constraint:

$$s_1 + s_2 + s_3 \leq 10$$

Additionally, all our investment values must be positive and less than \$10k (we could have set them with no upper bound and let the constraint equation take care of the upper bound). We'll rewrite our constraint again, so that we have:

$$10 - s_1 - s_2 - s_3 \geq 0$$

Let's formulate that in `minimize`.  We'll choose some values for our returns and variances, and set `gamma = 1` for now. Once again we are maximizing, so we'll need to minimize the opposite of our objective.

In [ ]:
r = [1.2, 1.8, 2.0]
v = [0.1, 0.2, 0.5]
v_total = 0.1
gamma = 1

def reward(s):
    positive = r[0]*s[0] + r[1]*s[1] + r[2]*s[2]
    risk = v[0]*s[0]**2 + v[1]*s[1]**2 + v[2]*s[2]**2 + v_total * sum(s)**2
    return positive - gamma*risk

ineq_cons = {'type': 'ineq', 
             'fun': lambda s: [10 - sum(s)],
             'jac': lambda s: [[-1.0, -1.0, -1.0]]}

minimize(lambda s: -reward(s), [3, 3, 3], method='SLSQP', constraints=ineq_cons, bounds=[(0,10), (0,10), (0,10)])

## Analytical optimization with constraints

The method of Lagrange multipliers is a method of finding the local minima or local maxima of a function subject to equality or inequality constraints. Let's look at it using a simple example. Imagine we would like to minimize cost which is the following function of x and y:

$$c = x^2 + 4y$$

under the following constraint:

$$x + y = 6$$

The Lagrange function in this example is:
$$\mathcal{L}(x, y, \lambda) = x^2 + 4y - \lambda(6 - x - y)$$
and now we take partial derivatives:
$$\frac{\partial \mathcal{L}}{\partial x} = 2x + \lambda = 0$$
$$\frac{\partial \mathcal{L}}{\partial y} = 4 + \lambda = 0$$
$$\frac{\partial \mathcal{L}}{\partial y} = x + y - 6 = 0$$

Anaytical solution: $x=2, y=4$
And numerically:

In [ ]:
def cost(xy):
    return xy[0]**2 + 4*xy[1]

def constraint(xy):
    return xy[0] + xy[1]

ineq_cons = {'type': 'eq', 
             'fun': lambda xy: [constraint(xy) - 6], 
             'jac': lambda xy: [[1, 1]]} 

results = minimize(cost, [1, 1], method='SLSQP', constraints=ineq_cons)

print('x = {:.3f}, y = {:.3f}, giving cost = {:.3f} and area = {:.3f}'
      .format(results.x[0], results.x[1], cost(results.x), area(results.x)))

*Copyright &copy; 2022 Pragmatic Institute. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*